In [1]:
from yfapi import YahooFinanceAPI, Interval
from datetime import datetime, timedelta
import psycopg2
from sqlalchemy import create_engine
from decouple import config

In [2]:
# Get database credentials, db path and twitter bearer token from env-file
dbuname = config('DBUNAME')
dbpwd = config('DBPWD')
host = config('HOST')
port = config('PORT')
dbname = config('DBNAME')

In [3]:
# instantiate the API and set the interval to daily data
dh = YahooFinanceAPI(Interval.DAILY)

In [4]:
# Investment instruments
inv_instruments = ['URTH', 'EXW1.DE', 'CSSMI.SW', 'EXXT.DE', 'GC=F', 'SI=F']
labels={'GC=F':'gold','SI=F':'silver','URTH':'msci_world', 'EXW1.DE':'euro_stoxx', 'CSSMI.SW':'smi', 'EXXT.DE':'nasdaq' }

In [7]:
# define function for fetching and loading the data

def fetch_load_data(instrument, start, end, method):
    
    while True:    
        #fetch data
        try:
            df = dh.get_ticker_data(instrument, start, end)
            table_name = 'table_yf_'+labels[instrument]
            print(table_name)
            df['Ins'] = instrument
            df['Ins_label'] = labels[instrument]
        except:
            print("Investment Instrument not available: " + instrument)
            break
        
                
        #load data
        if_ex_val = method
        
        try:
            conn_string = 'postgresql://'+dbuname+':'+dbpwd+'@'+host+':'+port+'/'+dbname
            engine = create_engine(conn_string)
            df.to_sql(table_name, conn_string, if_exists = if_ex_val, index=False) 
        except:
            print("Data load failed: " + table_name)   
            break
        break    
    

In [9]:
# Initial load of the historical data

# set timeframe
now = datetime(2022, 4, 6)
then = datetime(2021, 4, 5)

# set method of function df.to_sql()
method = 'append'

# execute function
for ins in inv_instruments:
    fetch_load_data(ins, then, now, method)

table_yf_msci_world
table_yf_euro_stoxx
table_yf_smi
table_yf_nasdaq
table_yf_gold
table_yf_silver


table_euro_stoxx


In [ ]:
# Fetch Data from database (to control the data load)

In [23]:
try: 
    conn = psycopg2.connect("host=dbyf1.c8vnebtzzltk.us-east-1.rds.amazonaws.com dbname=dbyf1 user=Christoph password=dbyahoo1")
except psycopg2.Error as e: 
    print("Error: Could not make connection to the Postgres database")
    print(e)
    
try: 
    cur = conn.cursor()
except psycopg2.Error as e: 
    print("Error: Could not get curser to the Database")
    print(e)
    
# Auto commit is very important
conn.set_session(autocommit=True)

In [24]:
try: 
    cur.execute("SELECT * FROM table_nasdaq;")
except psycopg2.Error as e: 
    print("Error: select *")
    print (e)

row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

(datetime.datetime(2021, 1, 4, 0, 0), 103.18, 103.300003, 100.879997, 101.040001, 101.040001, 154049, 'EXXT.DE', 'nasdaq')
(datetime.datetime(2021, 1, 5, 0, 0), 100.82, 101.699997, 100.32, 101.120003, 101.120003, 50853, 'EXXT.DE', 'nasdaq')
(datetime.datetime(2021, 1, 6, 0, 0), 99.43, 101.5, 98.830002, 101.300003, 101.300003, 48570, 'EXXT.DE', 'nasdaq')
(datetime.datetime(2021, 1, 7, 0, 0), 100.660004, 102.599998, 100.32, 102.5, 102.5, 52514, 'EXXT.DE', 'nasdaq')
(datetime.datetime(2021, 1, 8, 0, 0), 103.480003, 104.0, 103.199997, 103.620003, 103.620003, 45664, 'EXXT.DE', 'nasdaq')
(datetime.datetime(2021, 1, 11, 0, 0), 104.480003, 104.620003, 103.5, 104.019997, 104.019997, 45622, 'EXXT.DE', 'nasdaq')
(datetime.datetime(2021, 1, 12, 0, 0), 103.739998, 103.940002, 103.160004, 103.480003, 103.480003, 70719, 'EXXT.DE', 'nasdaq')
(datetime.datetime(2021, 1, 13, 0, 0), 103.279999, 104.120003, 102.879997, 104.120003, 104.120003, 22797, 'EXXT.DE', 'nasdaq')
(datetime.datetime(2021, 1, 14, 0, 

In [25]:
cur.close()
conn.close()